In [0]:
from pyspark.sql import SparkSession
import os
spark = SparkSession.builder.getOrCreate()

In [0]:
spark.sql("SHOW CATALOGS").show()


In [0]:
spark.sql("USE CATALOG adb_cliente_savana_prd")

In [0]:
# Criar schema da Bronze (caso não exista)
spark.sql("CREATE SCHEMA IF NOT EXISTS savana_elenir_bronze")

spark.sql("CREATE SCHEMA IF NOT EXISTS savana_elenir_silver")

spark.sql("CREATE SCHEMA IF NOT EXISTS savana_elenir_gold")


# Leitura dos dados

In [0]:
# MongoDB - leitura de transações
# -------------------------------
user_mongo = "savana"
password_mongo = os.environ.get("password_mongo") 
host_mongo = "52.146.23.117"
database_mongo = "transacao"
collection = "transacoes"

uri_mongo = f"mongodb://{user_mongo}:{password_mongo}@{host_mongo}:27017/{database_mongo}.{collection}?authSource=admin"

transacoes_df = (spark.read
    .format("com.mongodb.spark.sql.DefaultSource")
    .option("uri", uri_mongo)
    .option("database", database_mongo)
    .option("collection", collection)
    .load()
)

display(transacoes_df)


In [0]:
# MySQL - leitura de clientes
# -------------------------------
user_mysql = "root"
password_mysql = os.environ.get("password_mysql")  # <- Correto agora

url_mysql = "jdbc:mysql://52.146.23.117:3306/base_clientes?useSSL=false&serverTimezone=UTC"

clientes_df = (spark.read
    .format("jdbc")
    .option("driver", "com.mysql.cj.jdbc.Driver")
    .option("url", url_mysql)
    .option("dbtable", "Clientes")
    .option("user", user_mysql)
    .option("password", password_mysql)
    .load()
)

display(clientes_df)



In [0]:
transacoes_df.printSchema()
clientes_df.printSchema()

In [0]:
# Salvar transações no Unity Catalog (Bronze)
transacoes_df.write.mode("overwrite").format("delta") \
    .saveAsTable("adb_cliente_savana_prd.savana_elenir_bronze.transacoes")

# Salvar clientes no Unity Catalog (Bronze)
clientes_df.write.mode("overwrite").format("delta") \
    .saveAsTable("adb_cliente_savana_prd.savana_elenir_bronze.clientes")


#  Verificação de relacionamento
✅ Relação 1: 

ID_Cliente ↔ ID

Compatibilidade:

ID_Cliente (transações) é integer

ID (clientes) é integer

Conclusão: 

✅ Chave principal e estrangeira clássica.

⚠️ Relação 2:

 ID_Transacao ↔ Transacao

Ambos são inteiros, mas:

ID_Transacao na tabela de transações parece ser o identificador único da transação.


Transacao na tabela de clientes parece ser um campo auxiliar ou "última transação feita".